In [ ]:
from seq_loader.names_loader import TDTSDataset, collate_padded_batch_fn
from seq_loader.lightning import LitBaselineLSTM
from pathlib import Path
import torch
import lightning as L
from lightning.pytorch.loggers import TensorBoardLogger
import torch.utils.data as data

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
data_path = Path('/home/jovyan/data/tbts_data/')
filename = "tbts-names"
_NAME_CHARS_COL = str("name_chars")
tdts_dataset = TDTSDataset(
    tbts_path=data_path / f"{filename}.parquet",
    tbts_groupby_column="G",
    cat_columns=[_NAME_CHARS_COL],
    labels_path=data_path / f"{filename}-labels.parquet",
)

In [ ]:
embedding_dim = 8
col_embedding_dims = dict.fromkeys(tdts_dataset.cat_columns, embedding_dim)
embedding_dims = tdts_dataset.cat_col_embeddings_params(col_embedding_dims)
b_lstm = LitBaselineLSTM(embedding_dims=embedding_dims, h_size=32, n_classes=tdts_dataset.n_calsses)

In [ ]:
embedding_dim = 8
col_embedding_dims = dict.fromkeys(tdts_dataset.cat_columns, embedding_dim)
embedding_dims = tdts_dataset.cat_col_embeddings_params(col_embedding_dims)
b_lstm = LitBaselineLSTM(embedding_dims=embedding_dims, h_size=32, n_classes=tdts_dataset.n_calsses)

In [ ]:
train_set_size = int(len(tdts_dataset) * 0.8)
valid_set_size = len(tdts_dataset) - train_set_size

In [ ]:
seed = torch.Generator().manual_seed(42)
train_set, valid_set = data.random_split(tdts_dataset, [train_set_size, valid_set_size], generator=seed)

In [ ]:
train_loader = data.DataLoader(
    dataset=train_set,
    shuffle=True,
    batch_size=20,
    collate_fn=collate_padded_batch_fn,
    num_workers=2
)
valid_loader = data.DataLoader(
    dataset=valid_set,
    shuffle=True,
    batch_size=20,
    collate_fn=collate_padded_batch_fn,
    num_workers=2
)


In [ ]:
logger = TensorBoardLogger(save_dir=data_path / "tb_logs", name="besline_test")
trainer = L.Trainer(max_epochs = 500, accelerator="auto",
        devices=1, logger=logger)
trainer.fit(model=b_lstm, train_dataloaders=train_loader, val_dataloaders=valid_loader)

In [ ]:
! dirname $data_path

In [ ]:
! ls $data_path

In [ ]:
# tensorboard --host 0.0.0.0 --port 6006 --logdir=/home/jovyan/data/tbts_data/tb_logs